In [ ]:
# default_exp core

In [ ]:
#hide
from nbdev.showdoc import *

# Detector.py

> Set up a model with a set of trained weights.

Download the ultralytics YOLOv5 repository

In [ ]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
!git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f

Cloning into 'yolov5'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 5302 (delta 49), reused 55 (delta 27), pack-reused 5223
Receiving objects: 100% (5302/5302), 8.02 MiB | 2.75 MiB/s, done.
Resolving deltas: 100% (3618/3618), done.
/Users/philbrockman/coding/local-coding-projects/nbdev_template/yolov5
HEAD is now at 68211f7 FROM nvcr.io/nvidia/pytorch:20.10-py3 (#1553)


Imports & Installs

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  --use-feature=2020-resolver

import torch, time, utils
from utils.general import check_img_size, non_max_suppression
from utils.datasets import LoadStreams, LoadImages
from utils.general import scale_coords, xyxy2xywh
from utils.torch_utils import select_device
from utils.plots import plot_one_box
from models.experimental import attempt_load

In [ ]:
class Detector:
  """A wrapper for training loading saved YOLOv5 weights
  
  requirements:
    GPU enabled"""
  def __init__(self, weight_path, conf_threshold = .4, iou_threshold = .45, imgsz = 416, save_dir="save_dir", save_labeled_img = False, log_bboxes_as_txt = False, ):
    """ Constructor. I pulled the default numeric values above directly from the
    detect.py file. I added the option to save model output to both images and 
    to txt files

    Args:
      weight_path: the path to which the saved weights are stored
      conf_threshold: lower bound on the acceptable level of uncertainty for a 
                      bounding box
      iou_threshold: IoU helps determine how overlapped two shapes are.
      imgsz: resolution of image to process (assumes square)
      save_labeled_img: save a copy of the image with visibile bounding boxing
      log_bboxes_as_txt: create 1 (True) or 0 (False) txt files per bounding box 
    """
    self.weight_path = weight_path
    self.conf_threshold = conf_threshold
    self.iou_threshold = iou_threshold
    self.imgsz = imgsz
    self.device = select_device()
    self.model = attempt_load(self.weight_path, map_location=self.device)  # load FP32 model
    self.names = self.model.module.names if hasattr(self.model, 'module') else self.model.names
    self.imgsz = check_img_size(self.imgsz, s=self.model.stride.max())  # check img_size
    self.half = self.device.type != 'cuda'
    if self.half:
      self.model.half()  # to FP16

    self.save_labeled_img = save_labeled_img
    self.log_bboxes_as_txt = log_bboxes_as_txt
    self.save_dir = save_dir

  def process_image(self, source, save_unscuffed=False):
    """Runs on the model with pre-specified weights an input image. See original
    detect.py for more details

    Args:
      source: A string path to pre-specified weights for the model
      save_unscuffed: create copy of the pre-image

    Reurns:
      A JSON-serializable object encoding bounding box information
    """
    results = []
    img = torch.zeros((1, 3, self.imgsz, self.imgsz), device=self.device)  # init img
    _ = self.model(img.half() if self.half else img) if self.device.type != 'cpu' else None  # run once
    dataset = LoadImages(source, img_size=self.imgsz)

    for path, img, im0s, vid_cap in dataset:
      tmp = {}
      
      img = torch.from_numpy(img).to(self.device)
      img = img.half() if self.half else img.float()  # uint8 to fp16/32
      img /= 255.0  # 0 - 255 to 0.0 - 1.0
      if img.ndimension() == 3:
        img = img.unsqueeze(0)

      pred = self.model(img, augment=False)[0]
      pred = non_max_suppression(pred, self.conf_threshold, self.iou_threshold, agnostic=False)
      tmp["predictions"] = []
        
      for i, det in enumerate(pred):
        p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
        s += '%gx%g ' % img.shape[2:]  # out string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh

        if len(det):
          det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
          for c in det[:, -1].unique():
              n = (det[:, -1] == c).sum()  # detections per class
              s += f'{n} {self.names[int(c)]}s, '  # add to string
          
          #save original image
          tmp["unscuffed"] = im0

          for *xyxy, conf, cls in reversed(det):
            if self.log_bboxes_as_txt:  
              xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
              line = (cls, *xywh, conf) if True else (cls, *xywh)  # label format
              tmp["predictions"].append(('%g ' * len(line)).rstrip() % line)
              
              label = f'{self.names[int(cls)]} {conf:.2f}'
              plot_one_box(xyxy, im0, label=label, color=[0,0,200], line_thickness=5)

              # save image with bboxes drawn on top
              tmp["labeled"] = im0
        results.append(tmp)
    return tmp

In [ ]:
d = Detector("../21-2-25 1k-digits YOLOv5-weights.pt")

Fusing layers... 


In [ ]:
d.process_image("../example.png")

image 1/1 /Users/philbrockman/coding/local-coding-projects/nbdev_template/example.png: 

RuntimeError: "unfolded2d_copy" not implemented for 'Half'